In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import catboost

In [2]:
# Loading Data

train = pd.read_csv('data/train.csv')
songs = pd.read_csv('data/songs.csv')
song_labels = pd.read_csv('data/song_labels.csv')
test = pd.read_csv('data/test.csv')
save_for_later = pd.read_csv('data/save_for_later.csv')
dummy_submission = pd.read_csv('data/dummy_submission.csv')

In [3]:
from sklearn.model_selection import train_test_split
train_init, train_rem = train_test_split(train, train_size = 0.6, random_state=1023)

In [19]:
matrix_nan = train_init.pivot_table(index = 'customer_id', columns = 'song_id', values = 'score')

In [5]:
matrix_mean = matrix_nan - matrix_nan.mean()

In [6]:
matrix_mean.fillna(0, inplace = True)

In [7]:
corr_mat = np.corrcoef(matrix_mean.to_numpy(), rowvar = False)

C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [8]:
train_init['song_1'] = np.zeros(len(train_init.index.to_numpy()))

train_init['song_1_coeff'] = np.zeros(len(train_init.index.to_numpy()))

<ipython-input-8-dcbe057774a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1'] = np.zeros(len(train_init.index.to_numpy()))
<ipython-input-8-dcbe057774a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1_coeff'] = np.zeros(len(train_init.index.to_numpy()))


In [9]:
train_init['customer_id'].nunique()

14053

In [10]:
customer_groups = train_init.groupby('customer_id')

In [12]:
for i in range(len(train_init.index.to_numpy())):
    customer = train_init['customer_id'].iloc[i]
    song = train_init['song_id'].iloc[i]
    group = customer_groups.get_group(customer)
    customer_songs = group['song_id'].unique()
    customer_song_loc = customer_songs-1
    coeffs = corr_mat[customer_song_loc][:, song-1]
    indices = (-coeffs).argsort()[1]
    max_coeffs = coeffs[indices]
    nearest_songs = customer_songs[indices]
    train_init['song_1'].iloc[i] = nearest_songs
    
    train_init['song_1_coeff'].iloc[i] = max_coeffs
    
    if i%1000==0:
        print(i)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-12-2f1b139884b2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1'].iloc[i] = nearest_songs
<ipython-input-12-2f1b139884b2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1_coeff'].iloc[i] = max_coeffs


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


IndexError: index 1 is out of bounds for axis 0 with size 1

In [13]:
train_init['song_1_score'] = np.zeros(len(train_init.index.to_numpy()))

<ipython-input-13-bc2da0dd5b65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1_score'] = np.zeros(len(train_init.index.to_numpy()))


In [14]:
train_init

,customer_id,song_id,score,song_1,song_1_coeff,song_1_score
385157,I18663,634,5,761.0,0.056358,0.0
294688,L41976,24,4,25.0,0.174198,0.0
183843,N1008,2508,2,1616.0,0.091398,0.0
466122,H29702,200,4,88.0,0.030276,0.0
505943,I5773,2816,3,6136.0,0.100160,0.0
...,...,...,...,...,...,...
699722,J38184,155,5,0.0,0.000000,0.0
86949,J11904,2279,4,0.0,0.000000,0.0
619159,H31202,614,5,0.0,0.000000,0.0
31267,N31728,138,4,0.0,0.000000,0.0


In [16]:
for i in range(len(train_init.index.to_numpy())):
    song_1 = train_init['song_1'].iloc[i]
    
    customer = train_init['customer_id'].iloc[i]
    score_1 = matrix_nan[song_1].loc[customer]
    
    train_init['song_1_score'].iloc[i] = score_1
    if i%1000==0:
        print(i)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-16-bb4d8900cc39>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_init['song_1_score'].iloc[i] = score_1


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


KeyError: 0

In [18]:
train_init

,customer_id,song_id,score,song_1,song_1_coeff,song_1_score
385157,I18663,634,5,761.0,0.056358,5.0
294688,L41976,24,4,25.0,0.174198,3.0
183843,N1008,2508,2,1616.0,0.091398,1.0
466122,H29702,200,4,88.0,0.030276,2.0
505943,I5773,2816,3,6136.0,0.100160,2.0
...,...,...,...,...,...,...
699722,J38184,155,5,0.0,0.000000,0.0
86949,J11904,2279,4,0.0,0.000000,0.0
619159,H31202,614,5,0.0,0.000000,0.0
31267,N31728,138,4,0.0,0.000000,0.0
